# Beto Language Model

El modelo utilizado para resolver el problema planteado por la competencia está basado en BERT y su versión entrenada en español: [BETO](https://github.com/dccuchile/beto).

Este modelo tiene dos versiones preentrenadas: *cased* y *uncased*. En este caso el problema es *uncased* ya que las preguntas tienen un preprocesamiento que las llevó a *lowercase*. De todas formas decidí probar ambos modelos porque a veces sucede que la versión *cased* funciona mejor. 

Como primer paso antes de agregar un clasificador sobre Beto hice un finetuning sobre el lenguaje. 
Este paso fue realizado para adaptar a BETO al dominio del problema y que aprenda sobre él.

Esta etapa del entrenamiento es no supervisada por lo que utilicé todos los datos disponibles. 

El entrenamiento fue realizado durante 20 epochs.


# Código del entrenamiento

A continuación se encuentra el código utilizado para el entrenamiento del Language Modelling

In [ ]:
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from gensim.summarization.textcleaner import tokenize_by_word, AB_ACRONYM_LETTERS, tokenize

import torch
from torch.utils.data import TensorDataset, random_split

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

if torch.cuda.is_available():     
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
from transformers import *

import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
train = pd.read_csv('dataset/train.csv', sep='|')
test = pd.read_csv('dataset/test_santander.csv', sep=',')

In [ ]:

preguntas = pd.concat((train.Pregunta, test.Pregunta))
with open("dataset/train_embeddings.txt", "w") as file_object:
    for item in preguntas:
        file_object.write(f'{item}\n')

In [ ]:
max([len(x.split(' ')) for x in preguntas])

In [ ]:
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModelWithLMHead.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="dataset/train_embeddings.txt",
    block_size=48,
)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
# Calculate the number of samples to include in each set.
#train_size = int(0.9 * len(dataset))
#val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
train_dataset = dataset

In [ ]:
(len(train_dataset)/64)

In [ ]:
(len(train_dataset)/bs)*4*0.1

In [ ]:
from transformers import Trainer, TrainingArguments

MODEL_DIR = './BETOcased-20-epochs'

epochs = 20
bs = 64

training_args = TrainingArguments(
    output_dir=MODEL_DIR,
    overwrite_output_dir=True,
    #evaluate_during_training = True,
    learning_rate = 8e-5,
    logging_steps = int(len(train_dataset)/bs),
    max_steps = -1,
    num_train_epochs = epochs,
    warmup_steps=(len(train_dataset)/bs)*epochs*0.1,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs,
    do_train=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    #eval_dataset=val_dataset,
    prediction_loss_only=True,
)


In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()

In [ ]:

trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

In [ ]:
fill_mask = pipeline(
    "fill-mask",
    model=MODEL_DIR,
    tokenizer=MODEL_DIR
)

In [ ]:
fill_mask('Solámente con una tarjeta es posible [MASK] la transferencia.')